In [1]:
import warnings
import time
warnings.filterwarnings('ignore')
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from matplotlib import colors as mcolors
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
import pandas as pd
import sklearn as sk
from xgboost import XGBClassifier
from sklearn.preprocessing.imputation import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
import plotly.offline as py
import time
from sklearn.metrics import accuracy_score
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

# Wstepne przetwarzanie danych
Z racji, ze jest okolo 19GB danych testowych musialem podzielic je na mniejsze party 
(moje 32GB RAMu nie sa w stanie uciagnac wszystkiego jesli sie uwzgledni prace pythona,
ktory robi przewaznie kopie DataFramow do pamieci).  
Inny sposob to wczytywanie batchow do pamieci i operacja na nich bez uzycia posredniego zapisywania na dysku.
Jednak jezeli zastosuje sie opcje jaka jest ponizej, tylko pierwsze uruchomienie wymaga dwoch nastepnych komorek. Pozniejsze uruchomienia beda czytac juz z plikow "test_nr.csv

In [ ]:
dataset = pd.read_csv("C:\\Users\\Amadeusz\\Downloads\\all\\test_set.csv", 
                      dtype={"object_id":np.int64, "mjd":np.float64, "passband":np.int64,
                             "flux":np.float64, "flux_err":np.float64, "detected":np.int64})
print("Dataset loaded")

Podzial na 10 partow (text_n_part.csv)

In [ ]:
x = list(set(dataset["object_id"]))
n_parts = 10
dlugosc =  np.int(np.ceil(len(x)/n_parts))
for i, n in zip(range(0, len(x), dlugosc), range(1, 11)):
    start = time.time()
    dataset[dataset["object_id"].isin(x[i:i+dlugosc])].to_csv("..\\test_{}.csv".format(n))
    end = time.time()
    print("test_{}.csv saved - elapsed time: {}".format(n, end-start))
%xdel dataset
%xdel x

# Modele ML i statystyczne
Ogolnie czesc, ktora laduje modele z dysku o ile je mamy zapisane.
Jezeli nie, to na podstawie danych treningowych uzcy model i zapisuje na dysk.

Zawarty jest tez tutaj model wykrywania outlayers (wartosci odstajacych), w naszym przypadku novelty.
Pozniej bedzie potrzebny do klasyfikacji probek, ktore nie pasuja do schematu treningowego 
i nie powinny miec zadnej z wczesniejszych class/labels. Z racji tego, ze sa lepsze metody wykrywania dodatkowej klasy na ten moment zakomentowane.

In [3]:
import pickle
from sklearn.metrics import log_loss
try:
    # load the model from disk
    filename = '..\\LGBM_plus_XGB_model_log_loss.sav'
    clf = pickle.load(open(filename, 'rb'))
    print("Prediction model loaded.")
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#     # load the model from disk IsolationForest
#     filename = '..\\ISF.sav'
#     isf = pickle.load(open(filename, 'rb'))
#     print("Outlayer model loaded")
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
except FileNotFoundError as e:
    # load data ... (to be described)
    dataset = pd.read_csv("C:\\Users\\Amadeusz\\Downloads\\all\\training_set.csv")
    print("training dataset loaded")
    # load metadata, ... (to be described)
    meta_dataset = pd.read_csv('C:\\Users\\Amadeusz\\Downloads\\all\\training_set_metadata.csv')
    print("training metadataset loaded")
    column_names = {6: "class_6", 15: "class_15", 16: "class_16", 42: "class_42", 52: "class_52", 53: "class_53",
                    62: "class_62", 64: "class_64", 65: "class_65", 67: "class_67", 88: "class_88", 90: "class_90",
                    92: "class_92", 95: "class_95"}
    # change labels according to sample submission example
    meta_dataset["target"] = list(map(lambda name: column_names[name], meta_dataset["target"]))
    print("Class names for training metadataset changed.")
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# analysis part
    for key, val in {'u': 0, 'g': 1, 'r': 2, 'i': 3, 'z': 4, 'y': 5}.items():
        dataset[key] = (dataset['passband']-val).apply(np.bool).apply(np.logical_not).apply(np.int)
    print("Particular passbands extraction completed.")
    # convert mjd to unix time
    unix_time = (dataset['mjd']-40587)*86400
    #convert unix time to readable format
    time_ = pd.to_datetime(unix_time, unit='ms')
    dataset["hour"] = time_.dt.hour
    dataset["minute"] = time_.dt.minute
    dataset["second"] = time_.dt.second
    dataset["ms"] = time_.dt.microsecond
    print("MJD column decomposed to hour, minute, second and ms columns")
    import gc
    gc.enable()
    dataset['flux_ratio_sq'] = np.power(dataset['flux'] / dataset['flux_err'], 2.0)
    dataset['flux_by_flux_ratio_sq'] = dataset['flux'] * dataset['flux_ratio_sq']

    aggs = {
        'mjd': ['min', 'max', 'size'],
        'passband': ['min', 'max', 'mean', 'median', 'std'],
        'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
        'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
        'detected': ['mean'],
        'flux_ratio_sq':['sum','skew'],
        'flux_by_flux_ratio_sq':['sum','skew'],
        'r': ['sum'],
        'g': ['sum'],
        'i': ['sum'],
        'u': ['sum'],
        'z': ['sum'],
        'y': ['sum'],
        'hour': ['min', 'max', 'median'],
        'minute': ['min', 'max', 'median'],
        'second': ['min', 'max', 'median'],
        'ms': ['min', 'max', 'median'],
    }

    agg_train = dataset.groupby('object_id').agg(aggs)
    new_columns = [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    agg_train.columns = new_columns
    agg_train['hour_diff'] = agg_train['hour_max'] - agg_train['hour_min']
    agg_train['minute_diff'] = agg_train['minute_max'] - agg_train['minute_min']
    agg_train['second_diff'] = agg_train['second_max'] - agg_train['second_min']
    agg_train['ms_diff'] = agg_train['ms_max'] - agg_train['ms_min']
    agg_train['mjd_diff'] = agg_train['mjd_max'] - agg_train['mjd_min']
    agg_train['flux_diff'] = agg_train['flux_max'] - agg_train['flux_min']
    agg_train['flux_dif2'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_mean']
    agg_train['flux_w_mean'] = agg_train['flux_by_flux_ratio_sq_sum'] / agg_train['flux_ratio_sq_sum']
    agg_train['flux_dif3'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_w_mean']

    del agg_train['mjd_max'], agg_train['mjd_min']
    del agg_train['hour_max'], agg_train['hour_min']
    del agg_train['minute_max'], agg_train['minute_min']
    del agg_train['second_max'], agg_train['second_min']
    del agg_train['ms_max'], agg_train['ms_min']
    agg_train.head()

    del dataset
    gc.collect()
    print("New features computed and added to the train dataset.")
    meta_dataset['in_our_galaxy'] = meta_dataset['distmod'].apply(np.isnan).astype(int)
    print("New feature \"in_our_galaxy\" added.")
    meta_dataset = meta_dataset.fillna(0)
    print("\"distmod\" feature filled with zeros (our galaxy)")
    meta_dataset = meta_dataset.drop(['hostgal_specz'], axis=1)
    print("\"hostgal_specz\" feature removed.")
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    training_dataset = pd.merge(agg_train.reset_index(), meta_dataset)
    print("Training dataset merged.")
    
# w tym miejscu konczy sie preparowanie danych do uczenia modelu

    from sklearn.ensemble import VotingClassifier
    # train model and show score
    Y = training_dataset['target'] #wszystkie labels (gotowe do wrzucenia do modelu)
    X = training_dataset.drop(['target', 'object_id'], axis=1) #wszystkie dane treningowe (gotowe do wrzucenia do modelu)
    print("X and Y values prepared for training sequence.")
    start = time.time()
    models = [("XGBClassifier", XGBClassifier()), ("LGBMClassifier", LGBMClassifier())]
    model = VotingClassifier(models, voting='soft')
    clf = make_pipeline(StandardScaler(), model)
    clf.fit(X,Y)
    end = time.time()
    print("Scaling performed, XGB and LGBM model trained. Elapsed time: {}".format(end-start))

    # save model to disc
    filename = '..\\LGBM_plus_XGB_model_log_loss.sav'
    pickle.dump(clf, open(filename, 'wb'))
    print("Prediction model saved.")
    
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# IsolationForest part (novelty detection)
    # fit novelty detector model IsolationForest (~100 seconds computing time)
#     start = time.time()
#     isf = make_pipeline(StandardScaler(), IsolationForest())
#     isf.fit(np.ascontiguousarray(X.values
#     end = time.time()
#     print("IsolationForest trained. Elapsed time: {}".format(end-start))
    
#     # save isf model
#     filename = '..\\ISF.sav'
#     pickle.dump(isf, open(filename, 'wb'))
#     print("Outlayer (IsolationForest) estimator model saved.")
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

training dataset loaded
training metadataset loaded
Class names for training metadataset changed.
Particular passbands extraction completed.
MJD column decomposed to hour, minute, second and ms columns
New features computed and added to the train dataset.
New feature "in_our_galaxy" added.
"distmod" feature filled with zeros (our galaxy)
"hostgal_specz" feature removed.
Training dataset merged.
X and Y values prepared for training sequence.
Scaling performed, XGB model trained. Elapsed time: 37.87476086616516
Prediction model saved.
IsolationForest trained. Elapsed time: 1.4502766132354736
Outlayer (IsolationForest) estimator model saved.


# Predict part
Klasyfikacja probek, plus utworzenie nowej klasy w zaleznosci czy probka jest odstajaca czy nie.

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# def predictions(predicted_dataframe, object_id):
#     # ta metoda jak na razie spisywala sie najlepiej (nie wyprobowalem jej z lepszymi klasyfikatorami, warto sprobowac)
#     """Search rows with low prediction accuracy, if none of class was predicted higher then 0.6, classify as outlayer.
#     Add class_99 column for outlayers"""
#     columns = predicted_dataframe.columns.values.tolist()
#     start = time.time()
#     class_99 = np.any(predicted_dataframe[predicted_dataframe[columns] <= 0.6].apply(np.isnan), axis=1).apply(np.logical_not).apply(np.int)
#     predicted_dataframe["class_99"] = class_99
#     end = time.time()
#     print("After search, elapsed time: {}".format(end-start))
#     predicted_dataframe.insert(0,"object_id",object_id)
    
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Isolation Forest part
# def predictions_ISF(predicted_dataframe, object_id, data, isf):
#     """Use IsolationForest to classify if there is an outlayer, unseen label/class"""
#     columns = predicted_dataframe.columns.values.tolist()
#     n_samples = 1000000
#     l_data = len(data)
    
#     start = time.time()
#     class_99 =  np.array([])
#     for i in range(n_samples, l_data, n_samples): # IsolationForest cannot handle big datasets (32GB of RAM allocation is not enough)
#         #http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest.predict
#         class_99 = np.append(class_99, isf.predict(data[i-n_samples:i])) # IsolationForest predicts if sample is outlayer (-1 or inlayer 1)
#         print("Batch {} of {}, elapsed time: {}".format(i/n_samples, round(l_data/n_samples), time.time()-start))
#     class_99 = np.append(class_99, isf.predict(data[l_data-l_data%n_samples:]))
#     class_99[class_99 == 1] = 0 # change inlayers to 0 value
#     predicted_dataframe["class_99"] = -1*class_99 # append predictions with new class column
#     end = time.time()
    
#     print("After search, elapsed time: {}".format(end-start))
#     predicted_dataframe.insert(0,"object_id",object_id)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

def predictions_engi(predicted_dataframe, object_id, data):
    start = time.time()
    class_99 = np.ones(data.shape[0])
    for i in range(data.shape[1]):
        class_99 *= (1-data.iloc[:,i])
    predicted_dataframe['class_99'] = 0.14*class_99/np.mean(class_99)
    end = time.time()
    print("After search, elapsed time: {}".format(end-start))
    predicted_dataframe.insert(0,"object_id",object_id)
    
%xdel dataset
%xdel meta_dataset
%xdel imputed_meta_dataset
%xdel training_dataset
%xdel X_train
%xdel X_test
%xdel Y_train
%xdel Y_test
print("After memory relocation")

meta_dataset = pd.read_csv('C:\\Users\\Amadeusz\\Downloads\\all\\test_set_metadata.csv')
print("Test metadataset loaded.")
meta_dataset = meta_dataset.fillna(0)
print("All missing values filled with zeros (distmod)")
meta_dataset = meta_dataset.drop(['hostgal_specz'], axis=1)
print("\"hostgal_specz\" feature removed.")
meta_dataset['in_our_galaxy'] = meta_dataset['distmod'].apply(np.isnan).astype(int)
print("New feature \"in_our_galaxy\" added.")


for i in range(1,11):
    dataset = pd.read_csv("..\\test_{}.csv".format(i), index_col=0)
    print("Part {} test dataset loaded.".format(i))
    
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# analysis part
    for key, val in {'u': 0, 'g': 1, 'r': 2, 'i': 3, 'z': 4, 'y': 5}.items():
        dataset[key] = (dataset['passband']-val).apply(np.bool).apply(np.logical_not).apply(np.int)
    print("Particular passbands extraction completed.")
    # convert mjd to unix time
    unix_time = (dataset['mjd']-40587)*86400
    #convert unix time to readable format
    time_ = pd.to_datetime(unix_time, unit='ms')
    dataset["hour"] = time_.dt.hour
    dataset["minute"] = time_.dt.minute
    dataset["second"] = time_.dt.second
    dataset["ms"] = time_.dt.microsecond
    print("MJD column decomposed to hour, minute, second and ms columns")
    import gc
    gc.enable()
    dataset['flux_ratio_sq'] = np.power(dataset['flux'] / dataset['flux_err'], 2.0)
    dataset['flux_by_flux_ratio_sq'] = dataset['flux'] * dataset['flux_ratio_sq']

    aggs = {
        'mjd': ['min', 'max', 'size'],
        'passband': ['min', 'max', 'mean', 'median', 'std'],
        'flux': ['min', 'max', 'mean', 'median', 'std','skew'],
        'flux_err': ['min', 'max', 'mean', 'median', 'std','skew'],
        'detected': ['mean'],
        'flux_ratio_sq':['sum','skew'],
        'flux_by_flux_ratio_sq':['sum','skew'],
        'r': ['sum'],
        'g': ['sum'],
        'i': ['sum'],
        'u': ['sum'],
        'z': ['sum'],
        'y': ['sum'],
        'hour': ['min', 'max', 'median'],
        'minute': ['min', 'max', 'median'],
        'second': ['min', 'max', 'median'],
        'ms': ['min', 'max', 'median'],
    }

    agg_train = dataset.groupby('object_id').agg(aggs)
    new_columns = [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    agg_train.columns = new_columns
    agg_train['hour_diff'] = agg_train['hour_max'] - agg_train['hour_min']
    agg_train['minute_diff'] = agg_train['minute_max'] - agg_train['minute_min']
    agg_train['second_diff'] = agg_train['second_max'] - agg_train['second_min']
    agg_train['ms_diff'] = agg_train['ms_max'] - agg_train['ms_min']
    agg_train['mjd_diff'] = agg_train['mjd_max'] - agg_train['mjd_min']
    agg_train['flux_diff'] = agg_train['flux_max'] - agg_train['flux_min']
    agg_train['flux_dif2'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_mean']
    agg_train['flux_w_mean'] = agg_train['flux_by_flux_ratio_sq_sum'] / agg_train['flux_ratio_sq_sum']
    agg_train['flux_dif3'] = (agg_train['flux_max'] - agg_train['flux_min']) / agg_train['flux_w_mean']

    del agg_train['mjd_max'], agg_train['mjd_min']
    del agg_train['hour_max'], agg_train['hour_min']
    del agg_train['minute_max'], agg_train['minute_min']
    del agg_train['second_max'], agg_train['second_min']
    del agg_train['ms_max'], agg_train['ms_min']
    agg_train.head()

    del dataset
    gc.collect()
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    dataset_merged = pd.merge(agg_train.reset_index(), meta_dataset)
    print("Test dataset merged.")
    %xdel dataset
# tutaj konczy sie czesc z przygotowaniem danych do predykcji (testowych)
    object_id = dataset_merged["object_id"]
    print("Prediction started...")
    start = time.time()
    predicted_et = pd.DataFrame(clf.predict_proba(dataset_merged.iloc[:,1:].values), columns=clf.classes_)
    end = time.time()
    print("Part {} Predict Elapsed time: {}".format(i, end-start))
    
    # IsolationForest method
    print("Additional class prediction started...")
    strt = time.time()
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#IsolationForest Part
#     predictions_ISF(predicted_et, object_id, dataset_merged.iloc[:,1:].values, isf)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    predictions_engi(predicted_et, object_id, dataset_merged.iloc[:,1:])
    end = time.time()
    print("Additional class prediction finished. Elapsed time: {}".format(end-start))
#     Manual method
#     predictions(predicted_et, object_id)
    print("Save predictions to file started...")
    start = time.time()
    predicted_et.to_csv("..\\predicted_et_n_{}.csv".format(i))
    end = time.time()
    print("Part {} saved to CSV. Elapsed time: {}".format(i, end-start))
    %xdel dataset_merged
    %xdel predicted_et
    %xdel dataset_transformed

NameError: name 'dataset' is not defined
NameError: name 'imputed_meta_dataset' is not defined
NameError: name 'X_train' is not defined
NameError: name 'X_test' is not defined
NameError: name 'Y_train' is not defined
NameError: name 'Y_test' is not defined
After memory relocation
Test metadataset loaded.
All missing values filled with zeros (distmod)
"hostgal_specz" feature removed.
New feature "in_our_galaxy" added.
Part 1 test dataset loaded.
Particular passbands extraction completed.
MJD column decomposed to hour, minute, second and ms columns
Test dataset merged.
NameError: name 'dataset' is not defined
Prediction started...
Part 1 Predict Elapsed time: 45.36005759239197
IsolationForest prediction started...
After search, elapsed time: 38.096203327178955
IsolationForest finished. Elapsed time: 84.00627517700195
Save predictions to file started...
Part 1 saved to CSV. Elapsed time: 17.228618144989014
NameError: name 'dataset_transformed' is not defined
Part 2 test dataset loaded.
Pa

# Grupowanie
Scalanie partow w jedna calosc i zapis na dysk.

In [5]:
# merge all parts into one dataset
for i in range(1,11):
    start = time.time()
    if i == 1:
        dataset = pd.read_csv("..\\predicted_et_n_{}.csv".format(i), index_col=0)
    else:
        dataset = pd.concat([dataset, pd.read_csv("..\\predicted_et_n_{}.csv".format(i), index_col=0)])
    end = time.time()
    print("Group {} added. Elapsed time: {}".format(i, end-start))

dataset.to_csv("..\\LSST_LGBM_XGB.csv", index=False)
%xdel dataset

Group 1 added. Elapsed time: 6.890550374984741
Group 2 added. Elapsed time: 4.0390825271606445
Group 3 added. Elapsed time: 4.497156143188477
Group 4 added. Elapsed time: 4.541804075241089
Group 5 added. Elapsed time: 4.0001747608184814
Group 6 added. Elapsed time: 4.202584981918335
Group 7 added. Elapsed time: 3.741884708404541
Group 8 added. Elapsed time: 2.593755006790161
Group 9 added. Elapsed time: 2.712012529373169
Group 10 added. Elapsed time: 2.918860673904419
